In [52]:
from  pycirclize import *
from pycirclize.utils import load_example_tree_file, ColorCycler
from matplotlib.lines import Line2D
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import pandas as pd
import numpy as np

## Generatre phylogenic tree and show properties of sequence

In [105]:
tree_file = "CoV2_highph_salt_CDR3_tree.nwk"
circos, tv = Circos.initialize_from_tree(
    tree_file,
    r_lim=(10, 63),
    leaf_label_size=0,
    outer=True, 
    align_leaf_label=False, 
    ignore_branch_length=True,
    line_kws=dict(color="lightgrey", lw=1.0),
)


In [91]:
cdr3s = [x.replace("_","") for x in tv.leaf_labels]
pIs = [ProteinAnalysis(cdr3).isoelectric_point() for cdr3 in cdr3s]
gravys = [ProteinAnalysis(cdr3).gravy() for cdr3 in cdr3s]
lengths  = [len(cdr3) for cdr3 in cdr3s]

df = pd.DataFrame(
    dict(
        pI=pIs,
        phydrophobicity=gravys,
        lengths=lengths,
    ),
    index=tv.leaf_labels,
)

In [108]:
# Plot heatmaps (from `s1, s2, s3` column data)
sector = tv.track.parent_sector
track1 = sector.add_track((65, 75))
track1.heatmap(df["pI"].to_numpy(), cmap="coolwarm", show_value=False,vmin=min(pIs),vmax=max(pIs), rect_kws=dict(ec="grey", lw=0))
#circos.text("pI", r=track1.r_center, size=8, color="red")
circos.colorbar(
    bounds = (1, 0.6, 0.02, 0.3),
    vmin=min(pIs),
    vmax=max(pIs),
    cmap="coolwarm",
    orientation="vertical",
    colorbar_kws=dict(label="Isoelectric Point"))
track2 = sector.add_track((77, 87))
track2.heatmap(df["phydrophobicity"].to_numpy(),cmap="BrBG",vmin=min(gravys),vmax=max(gravys), show_value=False, rect_kws=dict(ec="grey", lw=0))
circos.colorbar(
    bounds=(1.2, 0.6, 0.02, 0.3),
    vmin=min(gravys),
    vmax=max(gravys),
    cmap="BrBG",
    orientation="vertical",
    colorbar_kws=dict(label="Hydrophobicity"))

bar_track = sector.add_track((89, 99), r_pad_ratio=0.1)
#bar_track.axis()
#bar_track.grid()
x = np.arange(0, tv.leaf_num) + 0.5
y = df["lengths"].to_numpy()
bar_track.bar(x, y, width=1,lw=0.5,color="orange")

circos.text('#CDR=667')


circos.savefig("result.pdf")